In [9]:
setwd("/Users/pranavkulkarni/SDM/Climate_Models_Arenaviruses")
library(raster)
library(terra)
library(ggplot2)

In [3]:
pers_vs_new <- function(species, quantiles = 10) {
  curr <- rast(paste0("./Data/Output/iterations/", species, "/current/foi_0.02.tif"))
  ssp2 <- rast(paste0("./Data/Output/iterations/", species, "/ssp2/foi_0.02.tif"))
  ssp5 <- rast(paste0("./Data/Output/iterations/", species, "/ssp5/foi_0.02.tif"))
  
  reclas <- function(foi, quantiles = 10) {
    hot_values <- values(foi)
    breaks = seq(min(hot_values, na.rm = TRUE), max(hot_values, na.rm = TRUE), length.out = quantiles+1)
    reclass_matrix <- cbind(
      breaks[-length(breaks)],  
      breaks[-1],               
      1:quantiles
    )
    hot <- classify(foi, reclass_matrix)
  }
  hot_curr <- reclas(curr)
  hot_ssp2 <- reclas(ssp2)
  hot_ssp5 <- reclas(ssp5)
  hot_pers_ssp2 <- ifel(hot_curr == 10 & hot_ssp2 == 10, 1,
                      ifel(hot_curr < 10 & hot_ssp2 == 10, 0, NA))
  hot_pers_ssp5 <- ifel(hot_curr == 10 & hot_ssp5 == 10, 1,
                        ifel(hot_curr < 10 & hot_ssp5 == 10, 0, NA))
  spot_pers_ssp2 <- as.points(hot_pers_ssp2)
  spot_pers_ssp5 <- as.points(hot_pers_ssp5)
  
  return(list(ssp2 = spot_pers_ssp2, ssp5 = spot_pers_ssp5))
  
}

In [4]:
zyg_list <- pers_vs_new("Z_brevicauda")
sig_list <- pers_vs_new("S_alstoni")
ccal_list <- pers_vs_new("C_callosus")
cmus_list <- pers_vs_new("C_musculinus")
cla_list <- pers_vs_new("C_laucha")
ofl_list <- pers_vs_new("O_flavescens")

In [11]:
ssp2 <- do.call("rbind", list(zyg_list[["ssp2"]],
                             sig_list[["ssp2"]],
                             ccal_list[["ssp2"]],
                             cmus_list[["ssp2"]],
                             cla_list[["ssp2"]],
                             ofl_list[["ssp2"]]))

In [12]:
ssp5 <- do.call("rbind", list(zyg_list[["ssp5"]],
                             sig_list[["ssp5"]],
                             ccal_list[["ssp5"]],
                             cmus_list[["ssp5"]],
                             cla_list[["ssp5"]],
                             ofl_list[["ssp5"]]))

In [15]:
writeVector(ssp2, './Data/Output/Hotspots/point_shapefiles/ssp2_persistent_vs_new.shp', overwrite = T)
writeVector(ssp5, './Data/Output/Hotspots/point_shapefiles/ssp5_persistent_vs_new.shp', overwrite = T)
